<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [7]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [8]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""

with open('../../data/hopsworkKey.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170584
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "china", "city": "chengdu", "street": "chengdu-jinbo-road", "aqicn_url": "https://api.waqi.info/feed/@10444", "latitude": "30.6861", "longitude": "104.0075"}'

In [7]:
today

datetime.date(2024, 11, 17)

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [9]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)
air_pm25rolling_fg = fs.get_feature_group(
    name='air_pm25rolling',
    version=2,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [10]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,13.0,china,chengdu,chengdu-jinbo-road,2024-11-19,https://api.waqi.info/feed/@10444


In [11]:
df = air_quality_fg.read()

# 假设 'date' 是时间列，按时间降序排序并提取最新两行
df_latest_3 = df.sort_values(by="date", ascending=False).head(3)

df_latest_3

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.02s) 


,date,pm25,country,city,street,url
2296,2024-11-19 00:00:00+00:00,13.0,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
2295,2024-11-18 00:00:00+00:00,13.0,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444
2294,2024-11-17 00:00:00+00:00,25.0,china,chengdu,chengdu-jinbo-road,https://api.waqi.info/feed/@10444


In [22]:
pm25_rolling_value = (df_latest_3.iloc[0]['pm25']+df_latest_3.iloc[1]['pm25']+df_latest_3.iloc[2]['pm25'])/3
new_df = aq_today_df.copy()  
new_df = new_df.rename(columns={ 'pm25':'pm25_rolling_mean'})
new_df.iloc[0, 0] =pm25_rolling_value
new_df['pm25_rolling_mean'] = new_df['pm25_rolling_mean'].astype('float64')
new_df

,pm25_rolling_mean,country,city,street,date,url
0,17.0,china,chengdu,chengdu-jinbo-road,2024-11-19,https://api.waqi.info/feed/@10444


In [23]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   pm25_rolling_mean  1 non-null      float64       
 1   country            1 non-null      object        
 2   city               1 non-null      object        
 3   street             1 non-null      object        
 4   date               1 non-null      datetime64[ns]
 5   url                1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 176.0+ bytes


In [14]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [15]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 30.75°N 104.0°E
Elevation 514.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,10.35,0.0,4.680000,22.619909,chengdu
1,2024-11-20,12.95,0.0,2.595997,56.309914,chengdu
2,2024-11-21,12.70,0.0,3.706427,29.054508,chengdu
3,2024-11-22,13.80,0.0,4.334974,131.633453,chengdu
4,2024-11-23,12.05,0.0,7.244860,63.435013,chengdu
5,2024-11-24,10.50,0.4,12.979984,3.179772,chengdu
6,2024-11-25,11.25,0.0,2.189795,9.462248,chengdu
7,2024-11-26,8.55,0.0,2.968636,284.036255,chengdu
8,2024-11-27,6.20,0.0,4.104631,254.744827,chengdu
9,2024-11-28,7.15,0.0,2.189795,350.537750,chengdu


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [16]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-19 18:19:08,019 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1170584/fs/1161287/fg/1347983


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1170584/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x27c55aa4f70>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 671787
         }
       },
       "result": {
         "observed_value": 12.999999046325684,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T05:19:08.000019Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations":

In [24]:
air_pm25rolling_fg.insert(new_df)

2024-11-19 18:21:46,439 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1170584/fs/1161287/fg/1358154


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_pm25rolling_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1170584/jobs/named/air_pm25rolling_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x27c55a955e0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_rolling_mean",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 680961
         }
       },
       "result": {
         "observed_value": 17.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T05:21:46.000439Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 

In [11]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-14 23:27:54,964 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1170584/fs/1161287/fg/1347984


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1170584/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1e95b520ac0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 671788
         }
       },
       "result": {
         "observed_value": 2.0364675521850586,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-14T10:27:54.000963Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
